In [ ]:
# In this notebook, we run inference and evaluation for each of the models on each of the other validation data sets, to assess the generalisation ability of each interface representation

In [ ]:
from pathlib import Path
from dataclasses import dataclass

import numpy as np

from src.interface_representation.interface_types import InterfaceType


In [ ]:
# First, assemble the 4 datasets which we want to use for evaluation
# We want the paths to the dataset, exluding the final dir which specifies the interface type

mu_sphere_1_base = Path('/Users/murray/Projects/multphase_flow_encoder/multiphase_flow_encoder/src/preprocessing/data/mu_spheres/spheres_mu_1.00')

mu_sphere_2_base = Path('/Users/murray/Projects/multphase_flow_encoder/multiphase_flow_encoder/src/preprocessing/data/mu_spheres/spheres_mu_2.00')

mu_sphere_25_base = Path('/Users/murray/Projects/multphase_flow_encoder/multiphase_flow_encoder/src/preprocessing/data/mu_spheres/spheres_mu_2.50')

hit_base = Path('/Users/murray/Projects/multphase_flow_encoder/multiphase_flow_encoder/src/preprocessing/data/patched_hit_experiment')

dataset_name_to_base = {
    'spheres_mu_100': mu_sphere_1_base,
    'spheres_mu_200': mu_sphere_2_base,
    'spheres_mu_250': mu_sphere_25_base,
    'hit': hit_base
}

dataset_interface_types = [
    'TANH_EPSILON0.0078125',
    'TANH_EPSILON0.015625',
    'TANH_EPSILON0.03125',
    'TANH_EPSILON0.0625',
    'TANH_EPSILON0.125',
    'TANH_EPSILON0.25',
    'HEAVISIDE',
    'SIGNED_DISTANCE_EXACT',
]

dataset_interface_types_to_interface_type = {
    'TANH_EPSILON0.0078125': InterfaceType.TANH_EPSILON,
    'TANH_EPSILON0.015625': InterfaceType.TANH_EPSILON,
    'TANH_EPSILON0.03125': InterfaceType.TANH_EPSILON,
    'TANH_EPSILON0.0625': InterfaceType.TANH_EPSILON,
    'TANH_EPSILON0.125': InterfaceType.TANH_EPSILON,
    'TANH_EPSILON0.25': InterfaceType.TANH_EPSILON,
    'HEAVISIDE': InterfaceType.HEAVISIDE,
    'SIGNED_DISTANCE_EXACT': InterfaceType.SIGNED_DISTANCE_EXACT,
}

dataset_interface_types_to_epsilon = {
    'TANH_EPSILON0.0078125': 0.0078125,
    'TANH_EPSILON0.015625': 0.015625,
    'TANH_EPSILON0.03125': 0.03125,
    'TANH_EPSILON0.0625': 0.0625,
    'TANH_EPSILON0.125': 0.125,
    'TANH_EPSILON0.25': 0.25,
    'HEAVISIDE': None,
    'SIGNED_DISTANCE_EXACT': None,
}

# Check all datasets are present, make sure directory exists

datasets = []

@dataclass
class Dataset():
    name: str
    interface_type: InterfaceType
    epsilon: float
    path: Path

for dataset_name, base in dataset_name_to_base.items():
    for interface_type in dataset_interface_types:
        path = base / interface_type
        assert path.exists(), f'{path} does not exist'
        
        it = dataset_interface_types_to_interface_type[interface_type]
        epsilon = dataset_interface_types_to_epsilon[interface_type]
        
        datasets.append(Dataset(dataset_name, it, epsilon, path))
        
# Given interface type and epsilon, map to a dict of dataset name to dataset path


def get_datasets_given_interface(interface_type, epsilon, datasets):
    matching_datasets = [d for d in datasets if d.interface_type == interface_type and d.epsilon == epsilon]
    return matching_datasets



In [ ]:
# Next, assemble the paths to the models which we want to evaluate (these are v24 and v25)

base_output_dir = Path('./../../output/lassen')
files_v25 = list(base_output_dir.glob('interfacial_ae_v25_*'))
files_v24 = list(base_output_dir.glob('interfacial_ae_v24_*'))

print(len(files_v25))
print(len(files_v24))

# For each one, we want to associate with:
# - The interface type
# - The dataset name

@dataclass
class TrainedModel():
    interface_type: InterfaceType
    epsilon: float
    dataset_name: str
    model_path: Path
    
trained_models = []

def extract_interface_type_v25(outdir):

    final_part = '_'.join(outdir.stem.split('_')[8:-1])

    final_part = final_part.replace('datadir', '')

    str_to_type = {
        'TANH_EPSILON00078125': InterfaceType.TANH_EPSILON,
        'TANH_EPSILON0015625': InterfaceType.TANH_EPSILON,
        'TANH_EPSILON003125': InterfaceType.TANH_EPSILON,
        'TANH_EPSILON00625': InterfaceType.TANH_EPSILON,
        'TANH_EPSILON0125': InterfaceType.TANH_EPSILON,
        'TANH_EPSILON025': InterfaceType.TANH_EPSILON,
        'HEAVISIDE': InterfaceType.HEAVISIDE,
        'SIGNED_DISTANCE_EXACT': InterfaceType.SIGNED_DISTANCE_EXACT,
    }

    return str_to_type[final_part]

def extract_epsilon_v25(outdir):
    final_part = '_'.join(outdir.stem.split('_')[8:-1])
    final_part = final_part.replace('datadir', '')
    
    str_to_epsilon = {
        'TANH_EPSILON00078125': 0.0078125,
        'TANH_EPSILON0015625': 0.015625,
        'TANH_EPSILON003125': 0.03125,
        'TANH_EPSILON00625': 0.0625,
        'TANH_EPSILON0125': 0.125,
        'TANH_EPSILON025': 0.25,
        'HEAVISIDE': None,
        'SIGNED_DISTANCE_EXACT': None,
    }
    
    return str_to_epsilon[final_part]

def extract_dataset_name_v25(outdir):
    part = '_'.join(outdir.stem.split('_')[5:8])
    part = part.replace('datadir', '')
    return part

def extract_model_path_v25(outdir):
    model_path = outdir / 'model-100.pt'
    assert model_path.exists()
    return model_path

def extract_interface_type_v24(outdir):

    final_part = '_'.join(outdir.stem.split('_')[5:-1])
    
    final_part = final_part.replace('datadir', '')

    str_to_type = {
        'TANH_EPSILON00078125': InterfaceType.TANH_EPSILON,
        'TANH_EPSILON0015625': InterfaceType.TANH_EPSILON,
        'TANH_EPSILON003125': InterfaceType.TANH_EPSILON,
        'TANH_EPSILON00625': InterfaceType.TANH_EPSILON,
        'TANH_EPSILON0125': InterfaceType.TANH_EPSILON,
        'TANH_EPSILON025': InterfaceType.TANH_EPSILON,
        'HEAVISIDE': InterfaceType.HEAVISIDE,
        'SIGNED_DISTANCE_EXACT': InterfaceType.SIGNED_DISTANCE_EXACT,
    }

    return str_to_type[final_part]

def extract_epsilon_v24(outdir):
    final_part = '_'.join(outdir.stem.split('_')[5:-1])
    final_part = final_part.replace('datadir', '')
    
    str_to_epsilon = {
        'TANH_EPSILON00078125': 0.0078125,
        'TANH_EPSILON0015625': 0.015625,
        'TANH_EPSILON003125': 0.03125,
        'TANH_EPSILON00625': 0.0625,
        'TANH_EPSILON0125': 0.125,
        'TANH_EPSILON025': 0.25,
        'HEAVISIDE': None,
        'SIGNED_DISTANCE_EXACT': None,
    }
    
    return str_to_epsilon[final_part]

def extract_dataset_name_v24(outdir):
    return 'hit'

def extract_model_path_v24(outdir):
    model_path = outdir / 'model-15.pt'
    assert model_path.exists()
    return model_path


for f in files_v24:
    interface_type = extract_interface_type_v24(f)
    dataset_name = extract_dataset_name_v24(f)
    model_path = extract_model_path_v24(f)
    epsilon = extract_epsilon_v24(f)
    
    trained_models.append(TrainedModel(interface_type, epsilon, dataset_name, model_path))
    
for f in files_v25:
    interface_type = extract_interface_type_v25(f)
    dataset_name = extract_dataset_name_v25(f)
    model_path = extract_model_path_v25(f)
    epsilon = extract_epsilon_v25(f)
    
    trained_models.append(TrainedModel(interface_type, epsilon, dataset_name, model_path))
    
print(len(trained_models))


In [ ]:
# Iterate over every model / dataset combination
# For each combination, run inference on the dataset, recording predictions and gt in dedicated results dir

from eval_utils import load_dataset, load_model, run_inference
import numpy as np
from tqdm import tqdm

for tm in tqdm(trained_models, desc='Evaluating all trained models'):
    
    matching_datasets = get_datasets_given_interface(tm.interface_type, tm.epsilon, datasets)
    
    assert len(matching_datasets) == 4
    
    for dataset_info in matching_datasets:
        print(f'Interface: {tm.interface_type}, {tm.epsilon}. Model trained on {tm.dataset_name}, evaluating on {dataset_info.name}')
        
        model = load_model(tm.model_path)
        
        if tm.dataset_name == 'hit':
            dataset = load_dataset(dataset_info.path, max_num_samples=25_000)
        else:
            dataset = load_dataset(dataset_info.path)
            
        gts, preds = run_inference(dataset, model, N=50)
        
        outname = f'{tm.interface_type}_{tm.epsilon}_{tm.dataset_name}_{dataset_info.name}.npz'
        outdir = Path('ood_results')
        np.savez_compressed(outdir / outname, gts=gts, preds=preds, epsilon=tm.epsilon, interface_type=tm.interface_type, train_dataset_name=tm.dataset_name, eval_dataset_name=dataset_info.name)
        
        
    
    
    

In [ ]:

# Now, let's compute dice scores for all these results
# Construct a large pandas DataFrame where we store: interface label (type + epsilon), train dataset, eval dataset, mean dice score

from eval_utils import get_phi_sharp, dice_coefficient
from fractions import Fraction
import pandas as pd
import matplotlib.pyplot as plt

result_rows = []


def float_to_fraction(float_num):
    fraction = Fraction(float(float_num)).limit_denominator()
    return f"{fraction.numerator}/{fraction.denominator}"


def get_interface_label(interface_type, epsilon):
    if interface_type == InterfaceType.TANH_EPSILON:
        return f'Tanh {float_to_fraction(epsilon)}'
    elif interface_type == InterfaceType.HEAVISIDE:
        return 'Sharp'
    elif interface_type == InterfaceType.SIGNED_DISTANCE_EXACT:
        return 'SDF'
    elif interface_type == InterfaceType.SIGNED_DISTANCE_APPROXIMATE:
        return 'SDF approx'
    else:
        raise ValueError('Unknown interface type')
    
    
def adjust_dataset_labels(dataset_name):
    if dataset_name == 'spheres_mu_100':
        return 'Spheres ($\\mu=1.0$)'
    elif dataset_name == 'spheres_mu_200':
        return 'Spheres ($\\mu=2.0$)'
    elif dataset_name == 'spheres_mu_250':
        return 'Spheres ($\\mu=2.5$)'
    elif dataset_name == 'hit':
        return 'Droplet in HIT'
    else:
        raise ValueError('Unknown dataset name')
    

for inference_result in Path('ood_results').glob('*.npz'):
    data = np.load(inference_result, allow_pickle=True)
    gts = data['gts']
    preds = data['preds']
    interface_type = data['interface_type']
    
    dice_scores = []
    
    for gt, pred in zip(gts, preds):
        gt_sharp = get_phi_sharp(gt, interface_type)
        pred_sharp = get_phi_sharp(pred, interface_type)
        
        dice = dice_coefficient(gt_sharp, pred_sharp)
        dice_scores.append(dice)
        
    mean_dice = np.nanmean(dice_scores)
    
    result_rows.append({
        'interface_label': get_interface_label(interface_type, data['epsilon']),
        'train_dataset': adjust_dataset_labels(data['train_dataset_name']),
        'eval_dataset': adjust_dataset_labels(data['eval_dataset_name']),
        'mean_dice': mean_dice
    })
    
df = pd.DataFrame(result_rows)

# Now, for each interface_label, show a heatmap for train_dataset vs eval_dataset

interface_labels = df['interface_label'].unique()

# Sort interface labels
def sort_key(interface_label):
    if interface_label == 'Sharp':
        return 0
    elif interface_label == 'SDF':
        return 100
    else:
        # Extract and sort by epsilon
        return float(interface_label.split(' ')[1].split('/')[0]) / float(interface_label.split(' ')[1].split('/')[1])
    
interface_labels = sorted(interface_labels, key=sort_key)
    
print(interface_labels)

subdfs = []

for interface_label in interface_labels:
    subdf = df[df['interface_label'] == interface_label]
    subdf = subdf.drop(columns='interface_label')
    subdf = subdf.pivot(index='train_dataset', columns='eval_dataset', values='mean_dice')
    
    subdfs.append((interface_label, subdf))
    

In [ ]:
df[df['interface_label'] == 'SDF']

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(8, 4), dpi=200, sharex=True, sharey=True)

for i, (interface_label, subdf) in enumerate(subdfs):
    ax = axs[i // 4, i % 4]
    ax.set_title(interface_label)
    im = ax.imshow(subdf, cmap='viridis', vmin=0.5, vmax=1)
    ax.set_xticks(range(len(subdf.columns)))
    ax.set_yticks(range(len(subdf.index)))
    ax.set_xticklabels(subdf.columns, rotation=90)
    ax.set_yticklabels(subdf.index)
    
axs[0, 0].set_ylabel('Train')
axs[1, 0].set_ylabel('Train')
axs[1, 0].set_xlabel('Test')
axs[1, 1].set_xlabel('Test')
axs[1, 2].set_xlabel('Test')
axs[1, 3].set_xlabel('Test')
    
# Create colorbar on RHS
cbar_ax = fig.add_axes([1, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label='Mean Dice Coefficient')
    
plt.tight_layout()
plt.show()